<a href="https://colab.research.google.com/github/rio1004666/Colab/blob/main/Ch04.%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%B6%84%EC%84%9D/2_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
날짜: 2021/11/23
이름: 강병화
내용: 영화 리뷰 감성분석 실습하기
"""

!pip install konlpy

In [ ]:
import pickle , re
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# 데이터 불러오기 
train_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/file/movie_review_train.txt')
test_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/file/movie_review_test.txt')
train_data

In [45]:
# 전처리 함수 정의 
okt = Okt()
stop_words = ['은','는','을','를','이','가','하','것','들','의','에게','에서','있','되','등','한']
def preprocessing(text):
  
  # 한글 텍스트 아닌것 제외 
  
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',str(text))
  
  # 문장 형태소 분석
  
  review_text = okt.morphs(review_text, stem=True)
  
  # 불용어 제거 
  
  word_review = [w for w in review_text if not w in stop_words]
  
  return word_review
clean_train_review = []
clean_test_review = []

# 훈련 데이터 전처리 
for review in train_data['document']:
  clean_train_review.append(preprocessing(review))
for review in test_data['document']:
  clean_test_review.append(preprocessing(review))
clean_train_review[:5]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '익살스럽다',
  '연기',
  '돋보이다',
  '영화',
  '스파이더맨',
  '늙다',
  '보이다',
  '하다',
  '커스틴',
  '던스트',
  '너무나도',
  '이쁘다',
  '보이다']]

In [ ]:
# 토큰화 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)
word_dic = tokenizer.word_index
word_dic

In [ ]:
# 패딩
train_inputs = pad_sequences(train_sequences,maxlen=10,padding='post') # 텍스트 10개짜리 패딩을 걸어서 학습시킴
test_inputs = pad_sequences(test_sequences, maxlen=10,padding='post')
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

print(train_inputs[0])
print(train_labels[0])


In [ ]:
# 모델생성
model = Sequential()
model.add(Embedding(30000,10)) # 30000개의 단어를 입력할 수 있는 층  10개의 노드로 시작 
model.add(LSTM(128))           # 순환신경망 - RNN모델에서 좀더 장기기억을 위한 순환 엣지를 하나 더 추가한 LSTM
model.add(Dense(1,activation='sigmoid')) # 출력노드는 하나이다. 
model.summary()

In [ ]:
# 모델설정

model.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['acc'])

In [ ]:
# 모델학습

model.fit(train_inputs,
          train_labels,
          epochs=10,
          batch_size=128,
          validation_split=0.2)

In [ ]:
# 모델평가 

result = model.evaluate(test_inputs,test_labels)
result


In [ ]:
#
def sentiment_predict(text):
  
  # 한글 텍스트 아닌것 제외 
  
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',str(text))
  
  # 문장 형태소 분석
  
  review_text = okt.morphs(review_text, stem=True)
  
  # 불용어 제거 
  
  word_review = [w for w in review_text if not w in stop_words]
  
  encoded = tokenizer.texts_to_sequences([word_review])
  padded = pad_sequences(encoded, maxlen=10, padding='post')
  score = float(model.predict(padded))
  return score

In [ ]:
while True:
  review = input('리뷰 입력: ')
  score = sentiment_predict(review)
  if score > 0.5:
    print('{:.2f}% 확률로 긍정 리뷰입니다.\n'.format(score*100))
  else:
    print('{:.2f}% 확률로 부정 리뷰입니다.\n'.format((1-score)*100))